In [1]:
!pip install git+https://github.com/wasserth/TotalSegmentator.git
!pip install --upgrade rt_utils
!totalseg_set_license -l aca_HZ08ZGEUAM3H3U
!pip install dcm2niix

  Cloning https://github.com/wasserth/TotalSegmentator.git to /tmp/pip-req-build-7sjcz2yq
  Running command git clone --filter=blob:none --quiet https://github.com/wasserth/TotalSegmentator.git /tmp/pip-req-build-7sjcz2yq
  Resolved https://github.com/wasserth/TotalSegmentator.git to commit b709255d7c8698029968bedd03d26a925275871d
  Installing build dependencies ..done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... one
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Using cached matplotlib-3.10.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) .

In [3]:
import pandas as pd
metadata = pd.read_excel("../Data/metadata_checked20250302_withImages.xlsx")
metadata

,plans_file_path,plans_plan_name,plans_plan_date,plans_reference_dose_name,plans_reference_dose,plans_approval,plans_CT_series,plans_CT_study,plans_patient_id,plans_patient_dob,...,dosimetrics_file_pattern,structures_file_path,structures_CT_series,structures_CT_study,structures_approval,structures_patient_id,structures_processing_timestamp,structures_file_pattern,available_structures,organized_path
0,../DICOM/172543/RP.172543.odbytnica.dcm,odbytnica,20240821,PTV1,50.0,APPROVED,1.2.246.352.71.2.846132777028.18433625.2024070...,1.2.246.352.76.1.846132777028.364035.241864723...,172543,19540507.0,...,RD*.dcm,../DICOM/172543/RS.172543.Auto__Miednica.0002.dcm,1.2.246.352.71.2.846132777028.18439874.2024071...,1.2.246.352.76.1.846132777028.364035.241864723...,APPROVED,172543,2025-02-28T04:26:56.506875,RS*.dcm,"gl, Odbytnica - PTV, podkladka, Jelita - PTV, ...",../DICOM_OrganizedCT/172543/series_5
1,../DICOM/304962/RP.304962.Odb +.dcm,Odb +,20230915,odb,50.0,APPROVED,1.2.246.352.71.2.846132777028.17504783.2023070...,1.3.12.2.1107.5.1.4.49619.30000023070307470284...,304962,19591026.0,...,RD*.dcm,../DICOM/304962/RS.304962.CT_1.0001.dcm,1.2.246.352.71.2.846132777028.17502857.2023070...,1.3.12.2.1107.5.1.4.49619.30000023070307470284...,APPROVED,304962,2025-02-28T04:26:56.506875,RS*.dcm,"gl, Odbytnica - PTV, podkladka, Jelita - PTV, ...",../DICOM_OrganizedCT/304962/series_3
2,../DICOM/363068/RP.363068.odb.dcm,odb,20230426,PTV1,60.0,APPROVED,1.2.246.352.71.2.846132777028.17189896.2023030...,1.3.12.2.1107.5.1.4.49619.30000023030207475281...,363068,19501105.0,...,RD*.dcm,../DICOM/363068/RS.363068.CT_1.0001.dcm,1.2.246.352.71.2.846132777028.17190337.2023030...,1.3.12.2.1107.5.1.4.49619.30000023030207475281...,APPROVED,363068,2025-02-28T04:26:56.506875,RS*.dcm,"gl, Odbytnica - PTV, podkladka, Jelita - PTV, ...",../DICOM_OrganizedCT/363068/series_3
3,../DICOM/371163/RP.371163.odb.dcm,odb,20230217,PTV1,50.0,APPROVED,1.2.246.352.71.2.846132777028.17032888.2022123...,1.3.12.2.1107.5.1.4.49619.30000022122007420785...,371163,19480114.0,...,RD*.dcm,../DICOM/371163/RS.371163.CT_1.0001.dcm,1.2.246.352.71.2.846132777028.17033413.2022123...,1.3.12.2.1107.5.1.4.49619.30000022122007420785...,APPROVED,371163,2025-02-28T04:26:56.506875,RS*.dcm,"gl, Odbytnica - PTV, Jelita - PTV, Rdzen, Jeli...",../DICOM_OrganizedCT/371163/series_3
4,../DICOM/377007/RP.377007.RAP1.dcm,RAP1,20190926,PTV 1,50.0,APPROVED,1.2.246.352.71.2.846132777028.14782935.2019080...,1.3.12.2.1107.5.1.4.49619.30000019080107404148...,377007,19470716.0,...,RD*.dcm,../DICOM/377007/RS.377007.CT_1.0001.dcm,1.2.246.352.205.4843651491561318283.7818544043...,1.3.12.2.1107.5.1.4.49619.30000019080107404148...,APPROVED,377007,2025-02-28T04:26:56.506875,RS*.dcm,"gl, PTV 1, PTV2-PTV1, Opuszka Pracia, Pecherz,...",../DICOM_OrganizedCT/377007/series_3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,../DICOM/479238/RP.479238.Odb+w H.dcm,Odb+w H,20240821,odb+w,NaN,APPROVED,1.2.246.352.71.2.846132777028.18418428.2024070...,1.3.12.2.1107.5.1.4.49619.30000024062807373600...,479238,19661116.0,...,RD*.dcm,../DICOM/479238/RS.479238.CT_1.0001.dcm,1.2.246.352.205.4705975484960778407.1209330727...,1.3.12.2.1107.5.1.4.49619.30000024062807373600...,APPROVED,479238,2025-02-28T04:26:56.506875,RS*.dcm,"gl, Odbytnica - PTV, Jelita - PTV, Rdzen, Jeli...",../DICOM_OrganizedCT/479238/series_3
238,../DICOM/479297/RP.479297.odbytnica1+.dcm,odbytnica1+,20240823,PTV1,54.0,APPROVED,1.2.246.352.71.2.846132777028.18418569.2024070...,1.2.246.352.76.1.846132777028.363653.241804718...,479297,19510411.0,...,RD*.dcm,../DICOM/479297/RS.479297.Auto__Miednica.0001.dcm,1.2.246.352.71.2.846132777028.18418464.2024070...,1.2.246.352.76.1.846132777028.363653.241804718...,APPROVED,479297,2025-02-28T04:26:56.506875,RS*.dcm,"gl, Odbytnica - PTV, Jelita - PTV, dod44, Rdze...",../DICOM_OrganizedCT/479297/series_3
239,../DICOM/480007/RP.480007.Odb+W.dcm,Odb+W,20240830,odb+w,NaN,APPROVED,1.2.246.352.71.2.8461327

In [ ]:
import os
import subprocess
import pandas as pd

# Load your metadata DataFrame
# Example: metadata = pd.read_csv('metadata.csv')

# Define the path to the log file that tracks completed directories
log_file = '../Logs/TotalSegmentator_processed_directories.log'

# Define the path to the summary CSV file
summary_csv = '../Data/TotalSegmentator_processing_summary.csv'

def load_processed_directories(log_file):
    """Load the list of processed directories from the log file."""
    if os.path.exists(log_file):
        with open(log_file, 'r') as file:
            processed = file.read().splitlines()
        print(f"Loaded {len(processed)} processed directories from log.")
    else:
        processed = []
        print("No log file found. Starting fresh.")
    return set(processed)

def save_processed_directory(log_file, directory):
    """Append a processed directory to the log file."""
    with open(log_file, 'a') as file:
        file.write(f"{directory}\n")
    print(f"Logged processed directory: {directory}")

def run_command(command):
    """Run a shell command and handle exceptions."""
    try:
        result = subprocess.run(command, check=True, capture_output=True, text=True, shell=True, executable='/bin/bash')
        print(f"Success: {result.stdout}")
        return True
    except subprocess.CalledProcessError as e:
        print(f"Error: {e.stderr}")
        return False

def convert_dicom_to_nifti(dicom_dir, output_dir):
    """Convert DICOM series to NIfTI format using dcm2niix."""
    os.makedirs(output_dir, exist_ok=True)
    command = f"source ~/miniconda3/etc/profile.d/conda.sh && conda activate rt && dcm2niix -z y -o {output_dir} {dicom_dir}"
    print(f"Executing DICOM to NIfTI conversion: {command}")
    return run_command(command)

def run_totalsegmentator(input_path, output_dir, output_type):
    """Run TotalSegmentator on the given input path."""
    os.makedirs(output_dir, exist_ok=True)
    command = f"source ~/miniconda3/etc/profile.d/conda.sh && conda activate rt && TotalSegmentator -i {input_path} -o {output_dir} -ta total -ot {output_type}"
    print(f"Executing TotalSegmentator: {command}")
    return run_command(command)

def update_summary(summary_csv, patient_id, dicom_input, nifti_output, dicom_seg_output, nifti_seg_output):
    """Update the summary DataFrame with the paths for each patient."""
    new_row = {
        'Patient_ID': patient_id,
        'DICOM_Input': dicom_input,
        'NIfTI_Output': nifti_output,
        'DICOM_Segmentation_Output': dicom_seg_output,
        'NIfTI_Segmentation_Output': nifti_seg_output
    }
    
    if os.path.exists(summary_csv):
        summary_df = pd.read_csv(summary_csv)
    else:
        summary_df = pd.DataFrame(columns=['Patient_ID', 'DICOM_Input', 'NIfTI_Output', 'DICOM_Segmentation_Output', 'NIfTI_Segmentation_Output'])

    # Convert new_row to a DataFrame and concatenate
    new_row_df = pd.DataFrame([new_row])
    summary_df = pd.concat([summary_df, new_row_df], ignore_index=True)

    summary_df.to_csv(summary_csv, index=False)
    print(f"Updated summary CSV for patient: {patient_id}")
    
# Load the list of processed directories
processed_directories = load_processed_directories(log_file)

# Iterate over each patient directory in the metadata
for patient_dir in metadata['organized_path']:
    patient_id = os.path.basename(os.path.dirname(patient_dir))
    if patient_dir in processed_directories:
        print(f"Skipping already processed directory: {patient_dir}")
        continue

    print(f"Processing directory: {patient_dir}")

    # Convert DICOM to NIfTI
    nifti_output_dir = os.path.join(patient_dir, 'nifti')
    if not convert_dicom_to_nifti(patient_dir, nifti_output_dir):
        print(f"Failed to convert DICOM to NIfTI for directory: {patient_dir}. Skipping.")
        continue

    # Identify the converted NIfTI file
    nifti_files = [f for f in os.listdir(nifti_output_dir) if f.endswith('.nii') or f.endswith('.nii.gz')]
    if not nifti_files:
        print(f"No NIfTI files found in {nifti_output_dir}. Skipping NIfTI segmentation.")
        continue
    nifti_file = os.path.join(nifti_output_dir, nifti_files[0])

    # Run TotalSegmentator in DICOM mode
    dicom_output_dir = f"{patient_dir}_total_dicom"
    if not run_totalsegmentator(patient_dir, dicom_output_dir, 'dicom'):
        print(f"Failed to run TotalSegmentator in DICOM mode for directory: {patient_dir}. Skipping.")
        continue

    # Run TotalSegmentator in NIfTI mode
    nifti_seg_output_dir = f"{patient_dir}_total_nifti"
    if not run_totalsegmentator(nifti_file, nifti_seg_output_dir, 'nifti'):
        print(f"Failed to run TotalSegmentator in NIfTI mode for file: {nifti_file}. Skipping.")
        continue

    # Log the processed directory
    save_processed_directory(log_file, patient_dir)

    # Update the summary DataFrame
    update_summary(summary_csv, patient_id, patient_dir, nifti_file, dicom_output_dir, nifti_seg_output_dir)


Loaded 1 processed directories from log.
Processing directory: ../DICOM_OrganizedCT/172543/series_5
Executing DICOM to NIfTI conversion: source ~/miniconda3/etc/profile.d/conda.sh && conda activate rt && dcm2niix -z y -o ../DICOM_OrganizedCT/172543/series_5/nifti ../DICOM_OrganizedCT/172543/series_5
Success: Chris Rorden's dcm2niiX version v1.0.20241211  GCC13.3.0 x86-64 (64-bit Linux)
Found 173 DICOM file(s)
Convert 173 DICOM as ../DICOM_OrganizedCT/172543/series_5/nifti/series_5_MIEDNICA_3,00_Qr40_S3_20240704131419_5b (512x512x173x1)
Conversion required 22.338141 seconds (3.906210 for core code).

Executing TotalSegmentator: source ~/miniconda3/etc/profile.d/conda.sh && conda activate rt && TotalSegmentator -i ../DICOM_OrganizedCT/172543/series_5 -o ../DICOM_OrganizedCT/172543/series_5_total_dicom -ta total -ot dicom
